### 기준금리 크롤링
- 한국은행 기준금리 추이
- 2022년 11월 24일 기준: 1999년부터 2022년 11월 24일까지 54개의 기준금리

- 함수 get_base_rate(i, file_path) -> baserate.csv로 저장

                -i : 기준금리 데이터 갯수
                
                -file_path : to_csv 저장 경로 e.g.) C:/Users/사용자/Desktop

In [50]:
def get_base_rate(i, file_path):

    import requests
    import datetime
    import pandas as pd
    import numpy as np
    from bs4 import BeautifulSoup

    base_url= "https://www.bok.or.kr/portal/singl/baseRate/list.do?dataSeCd=01&menuNo=200643"
    response = requests.get(base_url, 'html.parser')
    soup= BeautifulSoup(response.text, 'lxml')

    dict_list= []
    # 1999년부터 2022년 11월까지 54개의 기준금리
    for i in range(i):
        dict_list.append({
            'date': pd.to_datetime(datetime.datetime.strptime(soup.find_all('td')[3*i].text + soup.find_all('td')[3*i+1].text, '%Y%m월 %d일')),
            'RATE': float(soup.find_all('td')[3*i+2].text)
        })

    pd_result= pd.DataFrame(dict_list)

    # date range 일별 추가
    idx= pd.date_range(pd_result.date.min(), pd_result.date.max())
    idx= pd.Series(idx)

    # 없는 행에 Nan을 넣어 병합한 후 Nan값을 이전 값으로 메꾸기
    baserate= pd.concat([pd.DataFrame({'date': idx[~idx.isin(pd_result.date)], 'RATE': np.nan}),pd_result]).sort_values('date').reset_index(drop=True).ffill(axis=0)
    
    baserate = baserate.merge(baserate.assign(date = baserate.date+pd.Timedelta(days=31)), 
               on='date',
               how='left', suffixes=['', '_30days_ago'])

    baserate.dropna(inplace=True) # 빈값 제거
    
    # 한달 전 기준 금리 비교 값
    baserate['change'] = np.select(
        condlist=[(baserate.RATE > baserate.RATE_30days_ago), (baserate.RATE == baserate.RATE_30days_ago),], 
        choicelist=['up', 'same',],
        default='down'
    )
    

    baserate.to_csv(path_or_buf= file_path + '/baserate.csv' ,index=False)
    
    return baserate

In [1]:
from get_base_rate import get_base_rate

In [2]:
get_base_rate(54, '.')

,date,RATE,RATE_30days_ago,change
31,1999-06-06,4.75,4.75,same
32,1999-06-07,4.75,4.75,same
33,1999-06-08,4.75,4.75,same
34,1999-06-09,4.75,4.75,same
35,1999-06-10,4.75,4.75,same
...,...,...,...,...
8599,2022-11-20,3.00,3.00,same
8600,2022-11-21,3.00,3.00,same
8601,2022-11-22,3.00,3.00,same
8602,2022-11-23,3.00,3.00,same


### 콜금리
- 네이버 콜금리 데이터
- 2022년 11월 기준 page 623: 콜금리 데이터 2005년~

- 함수 get_call_rate(page, file_path) -> callrate.csv로 저장

                - page : 콜금리 데이터 페이지
                
                - file_path : to_csv 저장 경로 e.g.) C:/Users/사용자/Desktop

In [60]:
def get_call_rate(page, file_path):
    
    import requests
    import datetime
    import pandas as pd
    import numpy as np
    from bs4 import BeautifulSoup

    call_url= "https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CALL&page={}"

    call_list= []

    for i in range(1,page):
        call_list.append(BeautifulSoup(requests.get(call_url.format(i), 'html.parser').text,'lxml').find_all('td'))

    result_list= []

    for list in call_list:        
        for i in range(len(list)//4):
            result_list.append({
                'date': list[4*i].text.strip(),
                'RATE': float(list[4*i+1].text)
            })    
    pd_result2= pd.DataFrame(result_list)
    pd_result2.date= pd_result2.date.astype('datetime64')

    # 원하는 전체 date range
    idx2= pd.date_range(pd_result2.date.min(), pd_result2.date.max())
    idx2= pd.Series(idx2)

    # 없는 행에 Nan을 넣어 병합한 후 Nan값을 이전 값으로 메꾸기
    callrate= pd.concat([pd.DataFrame({'date': idx2[~idx2.isin(pd_result2.date)], 'RATE': np.nan}),pd_result2]).sort_values('date').reset_index(drop=True).ffill(axis=0)
    
    callrate = callrate.merge(callrate.assign(date = callrate.date+pd.Timedelta(days=31)), 
               on='date',
               how='left', suffixes=['', '_30days_ago'])
    callrate.dropna(inplace=True)

    # 한달 전 콜 금리 비교 값
    callrate['change'] = np.select(
        condlist=[(callrate.RATE > callrate.RATE_30days_ago), (callrate.RATE == callrate.RATE_30days_ago),], 
        choicelist=['up', 'same',],
        default='down'
    )

    callrate.to_csv(path_or_buf= file_path + '/callrate.csv' ,index=False)
    
    return callrate

In [3]:
from get_call_rate import get_call_rate

In [4]:
get_call_rate(623, ".")

,date,RATE,RATE_30days_ago,change
31,2005-06-09,3.28,3.29,down
32,2005-06-10,3.29,3.30,down
33,2005-06-11,3.29,3.30,down
34,2005-06-12,3.29,3.30,down
35,2005-06-13,3.28,3.29,down
...,...,...,...,...
6403,2022-11-19,3.02,3.07,down
6404,2022-11-20,3.02,3.13,down
6405,2022-11-21,3.04,3.10,down
6406,2022-11-22,3.07,3.10,down
